In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import metrics
import catboost as cb
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from mlxtend.preprocessing import minmax_scaling
# for Box-Cox Transformation
from scipy import stats
import xgboost as xgb
from sklearn.metrics import roc_auc_score

from sklearn import preprocessing
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings 

from sklearn import linear_model
warnings.filterwarnings('ignore')

In [47]:
train = pd.read_csv('/kaggle/input/alidata/final_train.csv')
test = pd.read_csv('/kaggle/input/alidata/final_test.csv')

In [48]:
submition = pd.read_csv('/kaggle/input/financialresilience1234/SampleSubmission.csv')
df = pd.read_csv('/kaggle/input/financialresilience1234/Train.csv')

In [49]:
len(df.target) == len(train)

True

In [50]:
y = df.target

In [51]:
train.columns

Index(['country', 'region', 'population', 'age', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5',
       'Q6', 'Q8', 'Q9', 'Q10a', 'Q10b', 'Q11', 'Q12', 'Q13a', 'Q14', 'Q15',
       'Q16', 'Q17a', 'Q17b', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25',
       'Q26', 'Q27', 'owns_mobile', 'Q28'],
      dtype='object')

In [52]:
train.drop(["Q2"],axis=1,inplace=True)
train.drop(["Q3"],axis=1,inplace=True)
train.drop(["Q4"],axis=1,inplace=True)
train.drop(["Q5"],axis=1,inplace=True)
train.drop(["Q8"],axis=1,inplace=True)
train.drop(["Q9"],axis=1,inplace=True)
train.drop(["Q20"],axis=1,inplace=True)
train.drop(["Q21"],axis=1,inplace=True)
train.drop(["Q23"],axis=1,inplace=True)
train.drop(["Q28"],axis=1,inplace=True)

In [53]:
test.drop(["Q2"],axis=1,inplace=True)
test.drop(["Q3"],axis=1,inplace=True)
test.drop(["Q4"],axis=1,inplace=True)
test.drop(["Q5"],axis=1,inplace=True)
test.drop(["Q8"],axis=1,inplace=True)
test.drop(["Q9"],axis=1,inplace=True)
test.drop(["Q20"],axis=1,inplace=True)
test.drop(["Q21"],axis=1,inplace=True)
test.drop(["Q23"],axis=1,inplace=True)
test.drop(["Q28"],axis=1,inplace=True)

In [54]:
train.drop(["owns_mobile"], axis=1, inplace=True)

ntrain = train.shape[0] # to be used to split train and test set from the combined dataframe

all_data = pd.concat((train, test)).reset_index(drop=True)

all_data["debit_card"]=all_data["Q1"]+all_data["Q6"]
all_data["got_savings"]=all_data["Q11"]+all_data["Q12"]+all_data["Q13a"]+all_data["Q22"]
all_data["borrowed"]=all_data["Q16"]+all_data["Q14"]+all_data["Q15"]+all_data["Q17a"]+all_data["Q17b"]
all_data["given"]=all_data["Q19"]
all_data["benefit"]=all_data["Q24"]+all_data["Q25"]+all_data["Q26"]

all_data["internet_access"]=all_data["Q10a"]+all_data["Q10b"]
all_data.drop(["Q10b","Q10a","Q27"], axis=1, inplace=True)

all_data.drop(["Q6","Q11","Q12","Q13a","Q22","Q16","Q14","Q15","Q17a","Q17b","Q19","Q24","Q25","Q26","Q1"], axis=1, inplace=True)

all_data.drop(["owns_mobile"], axis=1, inplace=True)

#all_data['ID']=all_IDs["ID"]
#all_data.drop(["internet_access"], axis=1, inplace=True)
all_data.drop(["region"], axis=1, inplace=True)


train = all_data[:ntrain]
test = all_data[ntrain:]

In [56]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [57]:
scaled_train_data=scaler.fit_transform(train, y=y)

In [2]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [3]:
from sklearn.metrics import accuracy_score

In [62]:
X = scaled_train_data
y = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [66]:
#test data scaling
scaled_test_data=scaler.transform(test)

In [67]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 42
    }

In [ ]:
from sklearn.metrics import roc_auc_score

In [78]:
def objective(space):
    clf=xgb.XGBClassifier(
                    tree_method='gpu_hist',
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_9bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    score = roc_auc_score(y_test, pred)
    print ("SCORE:", score)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [79]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

[05:20:40] WARNING: ../src/learner.cc:627:             
Parameters: { "colsample_9bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                 
0.7029698541413338                                     
[05:20:40] WARNING: ../src/learner.cc:627:                                        
Parameters: { "colsample_9bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                            
0.7027456001931458

In [80]:
SEED = 42

In [32]:
cross_valid_scores = {}

In [81]:
%%time
parameters = {
    'max_depth': [3, 5, 7, 9], 
    'n_estimators': [100,200,300,400],
    'learning_rate': [0.01, 0.05, 0.1]
}

model_xgb = xgb.XGBClassifier(
    random_state=SEED,
    tree_method='gpu_hist'
)

model_xgb = GridSearchCV(
    model_xgb, 
    parameters, 
    cv=5,
    scoring='roc_auc'
)

model_xgb.fit(X_train, y_train)
print('-----')
print(f'Best parameters {model_xgb.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_xgb.best_score_:.3f}'
)
cross_valid_scores['xgboost'] = model_xgb.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 400}
Mean cross-validated accuracy score of the best_estimator: 0.789
-----
CPU times: user 5min 33s, sys: 1.58 s, total: 5min 35s
Wall time: 5min 9s


In [82]:
%%time
parameters = {
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [7, 15, 31],
}

model_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    device =  "gpu"
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_lgbm.fit(
    X_train, 
    y_train
)

print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 31}
Mean cross-validated accuracy score of the best_estimator: 0.787
-----
CPU times: user 2min 50s, sys: 38.1 s, total: 3min 29s
Wall time: 2min 8s


In [83]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
CPU times: user 58.4 s, sys: 158 ms, total: 58.5 s
Wall time: 59.2 s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [84]:
rs_model.score(X_test, y_test)

0.23102844320774096

In [100]:
submition.target = rs_model.predict(test)

In [101]:
submition.to_csv('rsm_mode_sub.csv',index=False)

In [87]:
pd.DataFrame(cross_valid_scores, index=['cross_valid_score']).T

,cross_valid_score
xgboost,0.788579
lightgbm,0.786983


In [104]:
def create_submission(model, test, model_name,idd = submition.ID):
    y_pred_test = model.predict_proba(test)[:, 1]
    sub = pd.DataFrame(
        {
            'ID': idd, 
            'target': y_pred_test,
        }
        
    )
    sub.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [105]:
test_pred_xgboost = create_submission(
    model_xgb,  test, "xgboost_3_07:AM"
)
test_pred_lightgbm = create_submission(
    model_lgbm, test, "lightgbm_3_07:AM"
)

In [ ]:
from sklearn.naive_bayes import GaussianNB
# X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

In [ ]:
gnb.fit(X_train, y_train)
gnb.score(X_test,y_test)